### Problem 54

In the card game poker, a hand consists of five cards and are ranked, from lowest to highest, in the following way:

High Card: Highest value card.
One Pair: Two cards of the same value.
Two Pairs: Two different pairs.
Three of a Kind: Three cards of the same value.
Straight: All cards are consecutive values.
Flush: All cards of the same suit.
Full House: Three of a kind and a pair.
Four of a Kind: Four cards of the same value.
Straight Flush: All cards are consecutive values of same suit.
Royal Flush: Ten, Jack, Queen, King, Ace, in same suit.
The cards are valued in the order:
2, 3, 4, 5, 6, 7, 8, 9, 10, Jack, Queen, King, Ace.

If two players have the same ranked hands then the rank made up of the highest value wins; for example, a pair of eights beats a pair of fives (see example 1 below). But if two ranks tie, for example, both players have a pair of queens, then highest cards in each hand are compared (see example 4 below); if the highest cards tie then the next highest cards are compared, and so on.

Consider the following five hands dealt to two players:

|Hand|Player 1|Player 2|Winner|
|1|5H 5C 6S 7S KD|2C 3S 8S 8D TD|Player 2|
| |Pair of Fives|Pair of Eights|         |
|2|5D 8C 9S JS AC|2C 5C 7D 8S QH|Player 1|
| |Highest card Ace|Highest card Queen|  |
|3|2D 9C AS AH AC|3D 6D 7D TD QD|Player 2|
| |Three Aces|Flush with Diamonds|       |
|4|4D 6S 9H QH QC|3D 6D 7H QD QS|Player 1|
| |Pair of Queens|Pair of Queens|        |
| |Highest card Nine|Highest card Seven| |
|5|2H 2D 4C 4D 4S|3C 3D 3S 9S 9D|Player 1|
| |Full House|Full House|                |
| |With Three Fours|With Three Threes|   |
 	

The file, poker.txt, contains one-thousand random hands dealt to two players. Each line of the file contains ten cards (separated by a single space): the first five are Player 1's cards and the last five are Player 2's cards. You can assume that all hands are valid (no invalid characters or repeated cards), each player's hand is in no specific order, and in each hand there is a clear winner.

How many hands does Player 1 win?

In [1]:
import pandas as pd
from collections import Counter
from itertools import groupby
from collections import namedtuple
df = pd.read_csv('./poker.csv')

In [2]:
df.head()

,Hands
0,8C TS KC 9H 4S 7D 2S 5D 3S AC
1,5C AD 5D AC 9C 7C 5H 8D TD KS
2,3H 7H 6S KC JS QH TD JC 2D 8S
3,TH 8H 5C QS TC 9H 4D JC KS JS
4,7C 5H KC QH JD AS KH 4C AD 4S


In [3]:
Card = namedtuple('Card', ['rank', 'suit'])

In [12]:
Ace = 14
King = 13
Queen = 12
Jack = 11

In [5]:
def map_rank(rank_to_map):
    if rank_to_map == 'A':
        return Ace
    elif rank_to_map == 'K':
        return King
    elif rank_to_map == 'Q':
        return Queen
    elif rank_to_map == 'J':
        return Jack
    else:
        return int(rank_to_map)

In [6]:
def map_suit(suit_to_map):
    if suit_to_map == 'H':
        return 1
    elif suit_to_map == 'D':
        return 2
    elif suit_to_map == 'C':
        return 3
    else: 
        return 4

In [7]:
def map_card(card_to_map):
    return Card(rank = map_rank(card_to_map[0]), suit = map_suit(card_to_map[1]))

In [8]:
map_card('JD')

Card(rank=11, suit=2)

In [9]:
map_card('AS')

Card(rank=14, suit=4)

In [10]:
map_card('2C')

Card(rank=2, suit=3)

In [19]:
class Hand(object):
    
    def __init__(self, *args):
        self.hand = sorted(args, key=lambda c: -c.rank)
        self.duplicates = set()
        for card in self.hand:
            if card.rank in self.duplicates:
                continue
            _duplicates = 0
            for card2 in self.hand:
                if card == card2:
                    continue
                if card.rank == card2.rank:
                    _duplicates += 1
            self.duplicates[card.rank] = _duplicates
            
        
    def __cmp__(self, other):
        for rule in self.rules():
            print(rule + ' ',)
            rule = getattr(self, rule)
            val = rule(other)
            print(val)
            if val:
                return val
            
    def rules(self):
        return []
    
    def _straight(self):
        last_card_rank = False
        for card in self.hand:
            if not last_card_rank and last_card_rank != card.rank + 1:
                return False
            last_card_rank = card.rank
            else: 
                return True
            
    def _flush(self):
        last_card_suit = False
        for card in self.hand:
            if last_cards_suit and last_card_suit != card.rank:
                return False
            else:
                return True
            
    def _duplicates(self):

SyntaxError: invalid syntax (<ipython-input-19-d2d8b98c0f91>, line 36)

In [16]:
class PokerHand(Hand):
    def rules(self):
        rules = super(PokerHand, self).rules()
        rules.append('royal_flush')
        rules.append('straight_flush')
        rules.append('four_of_a_kind')
        rules.append('highest_card')
        return rules
    
    def royal_flush(self, other):
        h1 = self.hand[0] == Ace and self._flush() and self._straight()
        h2 = other.hand[0] == Ace and other._flush() and other._straight()
        if h1 and not h2:
            return 1
        elif h2 and not h1:
            return -1
        elif h1 and h2:
            return 0

    def straight_flush(self, other):
        h1 = self._flush() and self._straight()
        h2 = other._flush() and other._straight()
        if h1 and not h2:
            return 1
        elif h2 and not h1:
            return -1
        elif h1 and h2:
            return self.highest_card(other)
    
    def four_of_a_kind(self, other):
        pass
            
    def highest_card(self, other):
        for c1, c2 in zip(self.hand, other.hand):
            if c1.rank != c2.rank:
                return 1 if c1.rank > c2.rank else -1
            raise Exception('tie in highest_card')

NameError: name 'Hand' is not defined

In [70]:
h1 = PokerHand(Card(rank=5, suit=1), Card(rank=1, suit=1), Card(rank=6, suit=1))
h2 = PokerHand(Card(rank=6, suit=1), Card(rank=2, suit=1), Card(rank=6, suit=1))
h1.__cmp__(h2)

TypeError: 'method' object is not iterable